# Data Privacy Final Project
### Bradon Soucy and Sam Will

In [43]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [35]:
bioQuery = pd.read_csv("BioTIMEQuery_24_06_2021.csv", delimiter=',', encoding='ISO-8859-2', dtype={'PLOT': str}, usecols=[i for i in range(1,15)])
pd.set_option('display.max_columns', None)
bioQuery.head()

,STUDY_ID,DAY,MONTH,YEAR,SAMPLE_DESC,PLOT,ID_SPECIES,LATITUDE,LONGITUDE,sum.allrawdata.ABUNDANCE,sum.allrawdata.BIOMASS,GENUS,SPECIES,GENUS_SPECIES
0,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,22,47.4,-95.12,1.0,0.0,Acer,rubrum,Acer rubrum
1,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,23,47.4,-95.12,3.0,0.0,Acer,saccharum,Acer saccharum
2,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,24,47.4,-95.12,1.0,0.0,Acer,spicatum,Acer spicatum
3,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Medium,12,607,47.4,-95.12,12.0,0.0,Corylus,cornuta,Corylus cornuta
4,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Small,12,1911,47.4,-95.12,1.0,0.0,Populus,pinnata,Populus pinnata


In [144]:
# Creates the clean.csv which is the bioQuery dataset but only every 10th row is included
count = 0
clean = 0
firstL = True
wfile = open("clean.csv", 'w', encoding='ISO-8859-2')
with open("BioTIMEQuery_24_06_2021.csv", encoding='ISO-8859-2') as file:
    for line in file:
        if not firstL:
            if count % 10 == 0:
                wfile.write(line)
        else:
            firstL = False
            wfile.write(line)
        count+=1
wfile.close()
print(clean)

0


In [142]:
counts = bioQuery['YEAR'].value_counts()
cleanC = cBioQ['YEAR'].value_counts()

print(counts)
print(cleanC)

2003    345388
2004    343798
2000    337009
2002    334470
2001    326664
         ...  
1910        16
1889        14
1874         3
1900         3
1902         1
Name: YEAR, Length: 120, dtype: int64
2003    34505
2004    34376
2000    33687
2002    33494
2001    32678
        ...  
1918        4
1910        2
1889        2
1900        1
1874        1
Name: YEAR, Length: 119, dtype: int64


In [143]:
cBioQ = pd.read_csv("clean.csv", delimiter=',', encoding='ISO-8859-2', dtype={'PLOT': str}, usecols=[i for i in range(1,15)])
print(len(cBioQ))
cBioQ.head()

855224


,STUDY_ID,DAY,MONTH,YEAR,SAMPLE_DESC,PLOT,ID_SPECIES,LATITUDE,LONGITUDE,sum.allrawdata.ABUNDANCE,sum.allrawdata.BIOMASS,GENUS,SPECIES,GENUS_SPECIES
0,10,NaN,NaN,1984,47.400000_-95.120000_12_Control_0_Small,12,607,47.4,-95.12,7.0,0.0,Corylus,cornuta,Corylus cornuta
1,10,NaN,NaN,1984,47.400000_-95.120000_12_Treefall_0_Small,12,23,47.4,-95.12,2.0,0.0,Acer,saccharum,Acer saccharum
2,10,NaN,NaN,1984,47.400000_-95.120000_1_Control_0_Small,1,23,47.4,-95.12,4.0,0.0,Acer,saccharum,Acer saccharum
3,10,NaN,NaN,1984,47.400000_-95.120000_1_Control_Base_Small,1,607,47.4,-95.12,2.0,0.0,Corylus,cornuta,Corylus cornuta
4,10,NaN,NaN,1984,47.400000_-95.120000_1_Treefall_Base_Small,1,22,47.4,-95.12,6.0,0.0,Acer,rubrum,Acer rubrum
